# **Importing**

In [1]:
%cd /home/ai-05/Desktop/sdp

/home/ai-05/Desktop/sdp


In [1]:
import json
import os
import pathlib
import random
from datetime import datetime
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras_tuner import Hyperband, Objective, BayesianOptimization
from tensorflow.keras.callbacks import ModelCheckpoint

2023-04-20 11:32:25.603939: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-20 11:32:25.635879: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 11:32:26.082236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
gpu = tf.config.list_physical_devices('GPU')
print(gpu)
if gpu:
    tf.config.experimental.set_virtual_device_configuration(gpu[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-20 11:32:32.943362: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-20 11:32:32.960634: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-20 11:32:32.960779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [27]:
NEWMODELNAME = "InceptionV3-STBv1.6"
# BASEMODELNAME = "InceptionV3-STBv1.0"
BASEMODELNAME = "ResNet50-STBv1.0_18"

# **Data generator**

In [4]:
def ds_generator(path, batch_size=16, to_1_hot=False, shuffle=True, image_height=224, image_width=224, seed=123, num_classes=3):
    dir = pathlib.Path(path)


    ds = tf.keras.utils.image_dataset_from_directory(
        dir,
        seed=seed,
        image_size=(image_height, image_width),
        batch_size=batch_size,
        shuffle=shuffle)
    
    if to_1_hot:
        # Create a function to convert labels to one-hot encoding
        num_classes = num_classes
        def to_one_hot(x, y):
            num_classes = 3
            y_one_hot = tf.one_hot(y, num_classes)
            return x, y_one_hot
        ds = ds.map(to_one_hot)
    
    return ds

In [5]:
train_ds = ds_generator("/home/ai-05/Desktop/sdp/dataset", 16, True)
test_ds = ds_generator("/home/ai-05/Desktop/sdp/dcrpi-167", 16,True)

Found 63306 files belonging to 3 classes.
Found 167 files belonging to 3 classes.


2023-04-20 11:33:11.758440: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-20 11:33:11.758618: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-20 11:33:11.758690: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

# **Prep**

In [6]:
block_layer_index = {
    0: [1,8,'conv2d_94-96'],
    1: [8,30,'conv2d_96-103'],
    2: [30,55,'conv2d_104-112'],
    3: [55,87,'conv2d_112-119'],
    4: [87,-1,'conv2d_120-187']
    # 5: [51,61,'conv3_block2'],
    # 6: [61,71,'conv3_block3'],
    # 7: [71,81,'conv3_block4'],
    # 8: [81,93,'conv4_block1'],
    # 9: [93,103,'conv4_block2'],
    # 10: [103,113,'conv4_block3'],
    # 11: [113,123,'conv4_block4'],
    # 12: [123, 133,'conv4_block5'],
    # 13: [133,143,'conv4_block6'],
    # 14: [143,155,'conv5_block1'],
    # 15: [155,165,'conv5_block2'],
    # 16: [165, -1,'conv5_block3']
 }

In [7]:
print(block_layer_index)

{0: [1, 8, 'conv2d_94-96'], 1: [8, 30, 'conv2d_96-103'], 2: [30, 55, 'conv2d_104-112'], 3: [55, 87, 'conv2d_112-119'], 4: [87, -1, 'conv2d_120-187']}


In [22]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
img_size = (224,224) 


In [28]:
base_model = tf.keras.models.load_model(f"/home/ai-05/Desktop/sdp/Models/{BASEMODELNAME}")
# base_model = InceptionV3(input_shape=img_size + (3,),
#                          weights='imagenet',
#                          include_top=False)

In [29]:
base_model.input_shape

(None, 224, 224, 3)

# **Tuner**

In [10]:
tuner_path = f'Tuner/{BASEMODELNAME}'
num_trials = 30
tunable_indices = [0,1,2,3,4]

In [11]:
def retrieve_trials(tuner_path, num_trials, tunable_indices):
    """
        'Args': 
            tuner_path: str something similiar to 'Tuner/ResNet50-STBv1.0_15'
            num_trials: int (count them)
            tunable_indices: list(int) to tune them
        'Returns':
            a dict of dict in a panda DataFrame
            cols: the trials 0 1 2 ...
            rows: the name of the hps

    """
    hps_trials = {}
    for i in range(num_trials):
        trial_dir = f"{tuner_path}/HB_tuner/trial_00{i:02d}/"
        if not os.path.exists(trial_dir):
            os.makedirs(trial_dir)
        f = open(f"{tuner_path}/HB_tuner/trial_00{i:02d}/trial.json", 'r')
        hps_trial = json.load(f)
        hps_trials[i] = hps_trial['hyperparameters']['values']
        
    d = pd.DataFrame(hps_trials)
    d = d.fillna(-1)
    for i in range(len(d.columns)): d[i] = d[i].astype(int)

    name = lambda i: block_layer_index[i][2]
    # in tunable_indices and already in hps_trials
    names = [name(t) for t in tunable_indices if name(t) in d.index] 
    d.T[names].T

    return d

In [12]:
def comp(hps, d_i):
    # compares two dicts if they are equal 
    for k in hps:
        if k not in d_i.index or  hps[k] != d_i[k]: return False
        else: continue
    return True

In [11]:
tuner_path

'Tuner/InceptionV3-STBv1.0'

In [25]:
def build_model(hp, retrieve_trial=True):
    
    # resnet_base = model.layers[2]
    # resnet_base.trainable = False
    print(model.input_shape)
    name = lambda i: block_layer_index[i][2]
    names = list(map(name, tunable_indices))

    hps = {}
    for i,name_ in enumerate(names):
        hps[name_] = hp.Choice(name_, [True,False]) == True
    if retrieve_trial:
        d = retrieve_trials( tuner_path, num_trials, tunable_indices)
        exists = True
        while exists:
            checks = []
            for i in range(len(d.columns)):
                equal = comp(hps,d[i])
                checks.append(equal)
                if equal: break
            checks = np.array(checks)
            exists = np.any(checks) # If there is any True value returns True
            
            if exists:
                print(f"HP combination already exists with the following values:{hps}")
            else:print(f"HP combination is new.\n{hps}")
                
            # Flip the value of the boolean at the chosen key
            flip_key = random.choice(tunable_indices)
            flip_key = name(flip_key)
            hps[flip_key] = not hps[flip_key]

    for bli_k, bli_v in block_layer_index.items():
        print(bli_k)
        if bli_k in tunable_indices:
            for layer in base_model.layers[bli_v[0]:bli_v[1]]:
                layer.trainable = hps[name(bli_k)]
        
    base_model.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=[tf.metrics.Recall()])
    return base_model
    

In [14]:
from keras_tuner.engine.hyperparameters import HyperParameters

# Define the hyperparameters
hp = HyperParameters()

In [15]:
new_model = build_model(hp, False)

0
1
2
3
4


In [21]:
tuner = Hyperband(
    new_model,
    objective=Objective('val_recall', direction='max'),
    max_epochs=20,
    factor=3,
    overwrite=True,
    directory=f'Tuner/{NEWMODELNAME}',
    project_name='HB_tuner')

TypeError: Inputs to a layer should be tensors. Got '<keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7f4c29ab6f50>' (of type <class 'keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters'>) as input for layer 'model_1'.

In [ ]:
%pip install keras-tuner[bayesian]

In [25]:
tuner_path

'/home/ai-05/Desktop/sdp/Tuner/InceptionV3-STBv1.0'

In [30]:
tuner = BayesianOptimization(
    hypermodel=new_model,
    objective=Objective('val_recall', direction='max'),
    max_trials=50,
    seed=123
)


TypeError: Inputs to a layer should be tensors. Got '<keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7fa0734d3580>' (of type <class 'keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters'>) as input for layer 'model_1'.

: 

In [19]:
tuner.search_space_summary(extended=False)

NameError: name 'tuner' is not defined

In [26]:
log_dir = f"logs/{NEWMODELNAME}"
print(NEWMODELNAME)
# create a TensorBoard callback to visualize training metrics
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# # Create the TrialCheckpoint callback
# class MyModelCheckpoint (ModelCheckpoint):
#     def __init__(self, *args, **kwargs):
#         self.now = 0
#         super(MyModelCheckpoint, self).__init__(
#             filepath = self.get_chpt_dir(),
#             save_weights_only=False,
#             save_best_only=True,
#             save_freq='epoch',
#             verbose=1
#         )
        
    
#     def on_epoch_end(self, epoch, logs=None):
#         super(MyModelCheckpoint, self).on_epoch_end(epoch, logs)
#         self.now = datetime.now().strftime("%Y%m%d%H%M%S")
 
    
#     def get_chpt_dir(self):
#         return f"Checkpoint/{NEWMODELNAME}/{self.now}_"+"{epoch:02d}.h5"

InceptionV3-STBv1.6


In [27]:
tuner.search(train_ds, epochs=20, validation_data=test_ds,callbacks=[tensorboard_callback]) #MyModelCheckpoint(), 

INFO:tensorflow:Oracle triggered exit


# **Results (Tuner)**

In [28]:
tuner.results_summary(num_trials=30)

Results summary
Results in ./untitled_project
Showing 30 best trials
Objective(name="val_recall", direction="max")

Trial 17 summary
Hyperparameters:
conv1: 1
conv2_block1: 0
conv2_block2: 1
conv2_block3: 0
conv3_block1: 1
conv3_block2: 0
conv3_block3: 1
conv3_block4: 1
Score: 0.772455096244812

Trial 38 summary
Hyperparameters:
conv1: 0
conv2_block1: 0
conv2_block2: 1
conv2_block3: 1
conv3_block1: 1
conv3_block2: 1
conv3_block3: 0
conv3_block4: 1
Score: 0.772455096244812

Trial 32 summary
Hyperparameters:
conv1: 1
conv2_block1: 0
conv2_block2: 1
conv2_block3: 0
conv3_block1: 1
conv3_block2: 1
conv3_block3: 0
conv3_block4: 0
Score: 0.772455096244812

Trial 31 summary
Hyperparameters:
conv1: 0
conv2_block1: 0
conv2_block2: 1
conv2_block3: 0
conv3_block1: 1
conv3_block2: 0
conv3_block3: 1
conv3_block4: 0
Score: 0.772455096244812

Trial 30 summary
Hyperparameters:
conv1: 0
conv2_block1: 0
conv2_block2: 1
conv2_block3: 1
conv3_block1: 1
conv3_block2: 1
conv3_block3: 0
conv3_block4: 1
Score

In [ ]:
all_hps = tuner.get_best_hyperparameters(num_trials=30)

In [ ]:
test_ds = ds_generator("dcrpi-167", 16 , True,False)

In [ ]:
for i,hps in enumerate(all_hps):
    print(f"best trial - {i}")
    tuned_model = build_model(hps)
    tuned_model.evaluate(test_ds)

In [ ]:
model=build_model(all_hps[0]).save(f"Models/{NEWMODELNAME}")

In [ ]:
model

# **Prediction**

In [ ]:
model = tf.keras.models.load_model(f"Models/{NEWMODELNAME}")

In [ ]:
test_ds = ds_generator(path="dcrpi-167",batch_size=1,to_1_hot= False,shuffle=False)
preds = model.predict(test_ds)

In [ ]:
y_preds = np.argmax(preds, axis=1)

Y = []
for images, labels in test_ds:
    for label in labels:
        Y.append(label)

y_test = np.array(Y)

In [ ]:
y_preds

In [ ]:
np.floor(4.3)

In [ ]:
def show_images(i_s,images):
    mid = (len(images))**0.5

    nrows = int(np.ceil(mid))
    ncols = int(np.floor(mid))
    class_names = ['can','paper','plastic']
    plt.figure(figsize=(14, 14))
    # for images, labels in train_ds.take(1):
    for i in range(len(images)):
        ax = plt.subplot(nrows, ncols, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"P-{class_names[y_preds[i_s[i]]]} A-{class_names[y_test[i_s[i]]]}")
        plt.axis("off")

In [ ]:
class_names = ['can','paper','plastic']
images = []
i_s = []
for i, (image, label) in enumerate(test_ds):
    if y_test[i] != y_preds[i]:
        im = tf.squeeze(image)
        if i==0: 
            print(im.shape)
        images.append(im)
        i_s.append(i)
show_images(i_s,images)

In [ ]:
cm = tf.math.confusion_matrix(y_test,y_preds)
cm

In [ ]:
train_ds = ds_generator("dataset", 16, True)
test_ds = ds_generator("dcrpi-167", 16, True)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=[tf.metrics.Recall() ])

log_dir = f"logs/{NEWMODELNAME}_p"
tf.keras.losses.mean_absolute_error
# create a TensorBoard callback to visualize training metricsf
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_callback = ModelCheckpoint(
    filepath = f'Tuner/{NEWMODELNAME}_p/'+'{epoch:02d}.h5',
    save_weights_only=False,
    save_best_only=True,
    save_freq='epoch',
    verbose=1)

In [ ]:
history = model.fit(  train_ds, 
validation_data=test_ds,
callbacks=[checkpoint_callback,tensorboard_callback],
epochs=50)

In [ ]:
model.layers[-1].get_config()